# Bagging Classifier (from scratch) — Python

This project implements **bagging (bootstrap aggregating)** for a small binary classification dataset using a **custom decision tree** learner and **majority voting**.

**What’s included**
- Bootstrap sampling (multiple rounds)
- Decision tree training per round (custom splits)
- Ensemble prediction via majority vote
- Evaluation on a validation set (with analysis of results)

> Note: The dataset is a small “toy” dataset used in a course exercise to demonstrate ensemble learning mechanics.


## How to run

1. Open the notebook in Jupyter (or Google Colab).
2. Run cells top-to-bottom.
3. Review the printed bootstrap samples, learned trees, and ensemble voting results.


## What is Bagging?

Bagging means **Bootstrap Aggregating**. It is a way to combine several models so their predictions are more accurate.

### The Problem Bagging Solves

A single decision tree often fits the training data too closely. Even small changes in the data can create very different trees. Bagging helps by training several trees on different samples, which reduces this problem.

### How Bagging Works

1. **Bootstrap Sampling:** Randomly find N examples from the training data and allowing repeats.

2. **Train Base Classifiers:** Build decision trees for each bootstrap sample. Each tree is built differently because it uses different data.

3. **Aggregate Predictions:** Each tree votes for a class. The class with the most votes is the final prediction.

### Why It Works

By averaging the predictions from many trees, random mistakes tend to cancel out. Groups of trees tend to be more reliable at handling new data than just one tree.

In [1]:
import random

## Data

- 3 binary attributes: A, B, and C, either 0 or 1.
- 2 classes: + (positive) and - (negative).
- 10 training instances, which are used to build the trees.
- 5 validation instances, which are used to test.

In [2]:
# Training data: [A, B, C, Class]
TRAIN = [
    [0, 0, 0, '+'],  # 1
    [0, 0, 1, '+'],  # 2
    [0, 1, 0, '+'],  # 3
    [0, 1, 1, '-'],  # 4
    [1, 0, 0, '+'],  # 5
    [1, 0, 0, '+'],  # 6
    [1, 1, 0, '-'],  # 7
    [1, 0, 1, '+'],  # 8
    [1, 1, 0, '-'],  # 9
    [1, 1, 0, '-'],  # 10
]

# Validation data: [A, B, C, Class]
VAL = [
    [0, 0, 0, '+'],  # 11
    [0, 1, 1, '+'],  # 12
    [1, 1, 0, '+'],  # 13
    [1, 0, 1, '-'],  # 14
    [1, 0, 0, '+'],  # 15
]

## Helper Functions

A few helper functions are needed to handle common tasks before building trees.

### Bootstrap Sampling Function

This function makes a bootstrap sample using the training data.

In [3]:
def get_bootstrap_sample(data, seed):
    """Randomly pick rows with replacement"""
    random.seed(seed)
    n = len(data)
    picked = []
    for i in range(n):
        picked.append(random.randint(0, n - 1))

    sample = []
    for i in picked:
        sample.append(data[i])

    return sample, picked

### Counting Function

This function finds out how many positive (+) and negative (-) examples are in a dataset.

In [4]:
def count_plus_minus(data):
    """Count + and - in data"""
    plus = 0
    minus = 0
    for row in data:
        if row[3] == '+':
            plus = plus + 1
        else:
            minus = minus + 1
    return plus, minus

### Majority Vote Function

This function finds and returns the class that appears most often in a dataset.

In [5]:
def get_majority(data):
    """Return most common class"""
    plus, minus = count_plus_minus(data)
    if plus >= minus:
        return '+'
    return '-'

### Split Function

This function divides the data according to a specific feature value.

Decision trees split data based on feature values. This function divides the data into two groups: one where the feature is 0 and the other where it is 1, allowing for the building of the left and right subtrees.

In [6]:
def split_by_feature(data, feat, val):
    """Get rows where feature equals value"""
    result = []
    for row in data:
        if row[feat] == val:
            result.append(row)
    return result

## Building a Decision Tree

A decision tree sorts examples by asking questions about their features. Each step inside the tree checks one feature, and each end point gives a class prediction.

In [7]:
def build_tree(data, depth):
    """Build a simple decision tree"""
    plus, minus = count_plus_minus(data)

    # Stop if pure or too deep
    if minus == 0:
        return '+'
    if plus == 0:
        return '-'
    if depth >= 3:
        return get_majority(data)

    # Find best feature (0=A, 1=B, 2=C)
    best = None
    best_score = -1

    for feat in [0, 1, 2]:
        left = split_by_feature(data, feat, 0)
        right = split_by_feature(data, feat, 1)

        if len(left) == 0 or len(right) == 0:
            continue

        # Score = how many correct
        p0, m0 = count_plus_minus(left)
        p1, m1 = count_plus_minus(right)
        score = max(p0, m0) + max(p1, m1)

        if score > best_score:
            best_score = score
            best = feat

    if best is None:
        return get_majority(data)

    # Build subtrees
    left = split_by_feature(data, best, 0)
    right = split_by_feature(data, best, 1)

    return {
        'feat': best,
        0: build_tree(left, depth + 1),
        1: build_tree(right, depth + 1)
    }

### Prediction Function

After building a tree, we use it to classify new instances.

In [8]:
def predict(tree, row):
    """Use tree to predict class"""
    if tree == '+' or tree == '-':
        return tree

    feat = tree['feat']
    val = row[feat]
    return predict(tree[val], row)

### Tree Printing Function

This function prints the tree in a clear way, showing the if-then rules.

In [9]:
def print_tree(tree, indent=""):
    """Print tree as rules"""
    names = ['A', 'B', 'C']

    if tree == '+' or tree == '-':
        print(indent + "=> " + tree)
        return

    feat = tree['feat']
    print(indent + names[feat] + " = 0:")
    print_tree(tree[0], indent + "  ")
    print(indent + names[feat] + " = 1:")
    print_tree(tree[1], indent + "  ")

## Display the Data

The training and validation data from Figure 3.36.

In [10]:
print("TRAINING DATA:")
print("Inst  A  B  C  Class")
print("-" * 25)
for i in range(len(TRAIN)):
    r = TRAIN[i]
    print(str(i + 1).rjust(3) + "   " + str(r[0]) + "  " + str(r[1]) + "  " + str(r[2]) + "    " + r[3])

TRAINING DATA:
Inst  A  B  C  Class
-------------------------
  1   0  0  0    +
  2   0  0  1    +
  3   0  1  0    +
  4   0  1  1    -
  5   1  0  0    +
  6   1  0  0    +
  7   1  1  0    -
  8   1  0  1    +
  9   1  1  0    -
 10   1  1  0    -


In [11]:
print("VALIDATION DATA:")
print("Inst  A  B  C  Class")
print("-" * 25)
for i in range(len(VAL)):
    r = VAL[i]
    print(str(i + 11).rjust(3) + "   " + str(r[0]) + "  " + str(r[1]) + "  " + str(r[2]) + "    " + r[3])

VALIDATION DATA:
Inst  A  B  C  Class
-------------------------
 11   0  0  0    +
 12   0  1  1    +
 13   1  1  0    +
 14   1  0  1    -
 15   1  0  0    +


## Bagging Rounds

The bagging algorithm: create 5 decision trees, each trained on a different bootstrap sample.


In [12]:
NUM_ROUNDS = 5
trees = []

print("BAGGING ROUNDS")
print("=" * 50)

for r in range(1, NUM_ROUNDS + 1):
    print("\n--- Round " + str(r) + " ---")

    # Get bootstrap sample
    sample, picked = get_bootstrap_sample(TRAIN, seed=42 + r)

    # Show which instances were picked
    picked_nums = []
    for p in picked:
        picked_nums.append(p + 1)
    print("Bootstrap sample (instances): " + str(picked_nums))

    # Build tree
    tree = build_tree(sample, 0)
    trees.append(tree)

    # Show tree
    print("Tree " + str(r) + ":")
    print_tree(tree, "  ")

BAGGING ROUNDS

--- Round 1 ---
Bootstrap sample (instances): [1, 5, 3, 8, 6, 2, 8, 10, 8, 10]
Tree 1:
  B = 0:
    => +
  B = 1:
    A = 0:
      => +
    A = 1:
      => -

--- Round 2 ---
Bootstrap sample (instances): [7, 9, 9, 2, 3, 7, 4, 5, 1, 4]
Tree 2:
  B = 0:
    => +
  B = 1:
    A = 0:
      C = 0:
        => +
      C = 1:
        => -
    A = 1:
      => -

--- Round 3 ---
Bootstrap sample (instances): [5, 7, 8, 5, 2, 5, 6, 1, 2, 8]
Tree 3:
  B = 0:
    => +
  B = 1:
    => -

--- Round 4 ---
Bootstrap sample (instances): [2, 7, 1, 10, 10, 4, 9, 3, 10, 9]
Tree 4:
  A = 0:
    B = 0:
      => +
    B = 1:
      C = 0:
        => +
      C = 1:
        => -
  A = 1:
    => -

--- Round 5 ---
Bootstrap sample (instances): [6, 2, 7, 9, 8, 10, 6, 5, 9, 7]
Tree 5:
  B = 0:
    => +
  B = 1:
    => -


## Validation Results

In [13]:
print("Predictions from each tree:")
print("-" * 50)

# Store predictions
all_preds = []
for i in range(len(VAL)):
    all_preds.append([])

# Show each round's predictions
header = "Round |"
for i in range(len(VAL)):
    header = header + " I" + str(i + 11) + " |"
print(header)

for r in range(NUM_ROUNDS):
    line = "  " + str(r + 1) + "   |"
    for v in range(len(VAL)):
        pred = predict(trees[r], VAL[v])
        all_preds[v].append(pred)
        line = line + "  " + pred + " |"
    print(line)

print("-" * 50)

Predictions from each tree:
--------------------------------------------------
Round | I11 | I12 | I13 | I14 | I15 |
  1   |  + |  + |  - |  + |  + |
  2   |  + |  - |  - |  + |  + |
  3   |  + |  - |  - |  + |  + |
  4   |  + |  - |  - |  - |  - |
  5   |  + |  - |  - |  + |  + |
--------------------------------------------------


In [14]:
correct = 0
print("Final Results:")
print("-" * 50)

for v in range(len(VAL)):
    inst = VAL[v]
    true_class = inst[3]

    # Count votes
    plus = 0
    minus = 0
    for p in all_preds[v]:
        if p == '+':
            plus = plus + 1
        else:
            minus = minus + 1

    # Majority vote
    if plus > minus:
        final = '+'
    else:
        final = '-'

    # Check if correct
    if final == true_class:
        result = "CORRECT"
        correct = correct + 1
    else:
        result = "WRONG"

    print("Instance " + str(v + 11) + ": votes +" + str(plus) + "/-" + str(minus) +
          " => " + final + " (true: " + true_class + ") " + result)

print("\n" + "=" * 50)
print("ACCURACY: " + str(correct) + "/" + str(len(VAL)) +
      " = " + str(correct * 100 // len(VAL)) + "%")
print("=" * 50)

Final Results:
--------------------------------------------------
Instance 11: votes +5/-0 => + (true: +) CORRECT
Instance 12: votes +1/-4 => - (true: +) WRONG
Instance 13: votes +0/-5 => - (true: +) WRONG
Instance 14: votes +4/-1 => + (true: -) WRONG
Instance 15: votes +4/-1 => + (true: +) CORRECT

ACCURACY: 2/5 = 40%


## Summary

- Implemented bagging with 5 decision trees
- Each tree was trained on a bootstrap sample (10 instances sampled with replacement)
- Combined predictions using majority voting

### Results
- Validation accuracy: 40% (2 out of 5 correct)

### Why the Accuracy is Low

The low accuracy is not surprising since the validation data has examples that do not match the patterns found in the training data.

- **Instance 13** (A=1, B=1, C=0) is labeled **+** in validation, but in training, all examples with A=1 and B=1 are labeled **-** (instances 7, 9, 10). The model cannot learn to predict + for this pattern.

- **Instance 14** (A=1, B=0, C=1) is labeled **-** in validation, but in training, instance 8 with the same features is labeled **+**.

This shows that bagging can only learn from patterns present in the training data. It cannot fix differences between the training and validation data.

---

## Notes

- This notebook was originally created for a course assignment and has been lightly cleaned for portfolio use.
- The validation results are discussed in the analysis section; the dataset contains patterns that can limit achievable accuracy with this small setup.
